In [1]:
from pyspark.sql import SparkSession

In [2]:
# 1. Создаем сессию Spark
spark = SparkSession.builder \
    .appName("SocialNetwork") \
    .getOrCreate()

In [6]:
# 2. Загружаем данные из posts.csv в RDD
posts_rdd = spark.sparkContext.textFile("posts.csv")

In [8]:
# 3. Пропускаем первую строку (заголовок)
posts_rdd = posts_rdd.filter(lambda line: not line.startswith("Post_id,Post_Type,comments,likes"))

In [9]:
# Примеры строк в RDD
posts_rdd.take(5)

['101,Carousel,268,16382',
 '102,Reel,138,9267',
 '103,Reel,1089,10100',
 '104,Reel,271,6943',
 '105,Reel,145,17158']

In [10]:
# 4. Напишите функцию парсинга , которая будет преобразовывать каждую строку в кортеж Python. При этом: 
#    - Post_id преобразуйте в int. 
#    - comments преобразуйте в int. 
#    - likes преобразуйте в int.
#    - Post_Type оставьте как есть (строка).
def parse_post(line):
    parts = line.split(",")
    post_id = int(parts[0])
    post_type = parts[1]
    comments = int(parts[2])
    likes = int(parts[3])
    return (post_id, post_type, comments, likes)

In [11]:
# 5. Примените функцию парсинга к RDD
posts_parsed_rdd = posts_rdd.map(parse_post)

In [12]:
# Выводим первые 5 элементов для проверки
posts_parsed_rdd.take(5)

[(101, 'Carousel', 268, 16382),
 (102, 'Reel', 138, 9267),
 (103, 'Reel', 1089, 10100),
 (104, 'Reel', 271, 6943),
 (105, 'Reel', 145, 17158)]

In [13]:
# 6. Посчитайте общее количество постов в датасете
total_posts = posts_parsed_rdd.count()
print(f"Total posts: {total_posts}")

Total posts: 39


In [14]:
# 7. Посчитайте общее количество лайков по всем постам
total_likes = posts_parsed_rdd.map(lambda x: x[3]).reduce(lambda a, b: a + b)
print(f"Total likes: {total_likes}")

Total likes: 707567


In [15]:
# 8. Посчитайте общее количество комментариев по всем постам
total_comments = posts_parsed_rdd.map(lambda x: x[2]).reduce(lambda a, b: a + b)
print(f"Total comments: {total_comments}")

Total comments: 8387


In [16]:
# 9. Определите среднее количество лайков на один пост.
# Округлите результат до одного знака после запятой
average_likes = round(total_likes / total_posts, 1)
print(f"Average likes per post: {average_likes}")

Average likes per post: 18142.7


In [17]:
# 10. Определите среднее количество комментариев на один пост.
# Округлите результат до одного знака после запятой
average_comments = round(total_comments / total_posts, 1)
print(f"Average comments per post: {average_comments}")

Average comments per post: 215.1


In [19]:
# 11. Посчитайте количество постов каждого типа
post_type_counts = posts_parsed_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)
print("Post type counts:")
for post_type, count in post_type_counts.collect():
    print(f"{post_type}: {count}")

Post type counts:
Carousel: 14
Reel: 14
Image: 11


In [20]:
# 12. Для каждого типа поста рассчитайте среднее количество лайков.  Округлите результат до одного знака после запятой.
average_likes_by_type = posts_parsed_rdd.map(lambda x: (x[1], (x[3], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: round(x[0] / x[1], 1))
print("Average likes by post type:")
for post_type, avg_likes in average_likes_by_type.collect():
    print(f"{post_type}: {avg_likes}")

Average likes by post type:
Carousel: 20814.3
Reel: 15156.6
Image: 18543.2


In [21]:
# 13. Для каждого типа поста рассчитайте среднее количество комментариев.  Округлите результат до одного знака после запятой.
average_comments_by_type = posts_parsed_rdd.map(lambda x: (x[1], (x[2], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: round(x[0] / x[1], 1))
print("Average comments by post type:")
for post_type, avg_comments in average_comments_by_type.collect():
    print(f"{post_type}: {avg_comments}")

Average comments by post type:
Carousel: 174.3
Reel: 316.7
Image: 137.5


In [22]:
# 14. Найдите Топ-5 постов с наибольшим количеством лайков. Выведите их Post_id, Post_Type , likes.
top_5_liked_posts = posts_parsed_rdd.takeOrdered(5, key=lambda x: -x[3])
print("Top 5 liked posts:")
for post in top_5_liked_posts:
    print(f"Post_id: {post[0]}, Post_Type: {post[1]}, Likes: {post[3]}")

Top 5 liked posts:
Post_id: 125, Post_Type: Carousel, Likes: 79000
Post_id: 131, Post_Type: Carousel, Likes: 59716
Post_id: 133, Post_Type: Carousel, Likes: 58485
Post_id: 132, Post_Type: Image, Likes: 53254
Post_id: 122, Post_Type: Image, Likes: 50523


In [23]:
# 15. Найдите Топ-5 постов с наибольшим количеством комментариев. Выведите их Post_id, Post_Type , comments.
top_5_commented_posts = posts_parsed_rdd.takeOrdered(5, key=lambda x: -x[2])
print("Top 5 commented posts:")
for post in top_5_commented_posts:
    print(f"Post_id: {post[0]}, Post_Type: {post[1]}, Comments: {post[2]}")

Top 5 commented posts:
Post_id: 103, Post_Type: Reel, Comments: 1089
Post_id: 109, Post_Type: Reel, Comments: 884
Post_id: 123, Post_Type: Reel, Comments: 555
Post_id: 131, Post_Type: Carousel, Comments: 507
Post_id: 125, Post_Type: Carousel, Comments: 466
